## Reflection on Day 21:
- Part 1 was quite chill, probably quite a few things I could have worded more succintly, I really didn't need to do all this weird minus 1 stuff to deal with modulus, could have worded a much simpler function
- However, part 2 was a real bitch until I realized that a lot of these "world states" were similar, and could be aggregated in a dictionary
- This was still hard until I learned the key fundamental difference between a tuple and a list, increasing my ability to work with dictionaries and using complex states as keys
- Essentially, a tuple is an array that contains multiple chunks of data. Because these values are fixed, it is possible to hash them and place them into a dictionary. A list is not fixed, rather it is an array of memory pointers to values. Thus, you can edit a list (by editing the value at that point of memory), and thus, you can't really hash a list (since the memory values are absolutely unique, and if you tried to sub in an "identical" list or tuple, you wouldn't get the same thing.
- Despite the cleaness of the solution in my head, it's still a little messy to read, and I can see a couple of code duplications. Looking at it now, it might have been smarter to declare separate functions for these sections

In [30]:
p1_position = 6
p2_position = 2

In [31]:
p1_position -= 1
p2_position -= 1

In [32]:
p1_score = 0
p2_score = 0

In [33]:
dice_state = 1

In [34]:
def take_a_turn():
    steps = 0
    for i in range(3):
        steps += dice_state
        dice_state += 1
    p1_position += steps
    p1_position = p1_position%10
    p1_score += p1_position + 1
    
    steps = 0
    for i in range(3):
        steps += dice_state
        dice_state += 1
    p2_position += steps
    p2_position = p2_position%10
    p2_score += p2_position + 1

In [35]:
count = 0
while p1_score <1000 and p2_score < 1000:
    steps = 0
    for i in range(3):
        steps += dice_state
        dice_state += 1
    p1_position += steps
    p1_position = p1_position%10
    p1_score += p1_position + 1
    count +=3
    
    if p1_score>=1000:
        break
    
    steps = 0
    for i in range(3):
        steps += dice_state
        dice_state += 1
    p2_position += steps
    p2_position = p2_position%10
    p2_score += p2_position + 1
    
    count += 3

In [36]:
count

1005

In [37]:
p1_score

1006

In [38]:
p2_score

922

In [39]:
count*min(p1_score,p2_score)

926610

In [89]:
p1_position = 6
p2_position = 2
p1_score = 0
p2_score = 0
current_turn = 'p1'

In [90]:
current_state = (p1_position, p2_position, p1_score, p2_score, current_turn)

In [91]:
worlds = {current_state:1}
worlds

{(6, 2, 0, 0, 'p1'): 1}

Because we can't possibly hope to handle each different world, we need to handle everything under the structure of a "world state" and how many worlds are under that state.

Because there will end up being many ways to end up in the same state, this can help save us a lot of memory and time.

The rough flow is that for each world, we will roll 3 dice (thus generating 27 new worlds) and then group them up into a dictionary object which hashes each state to the number of worlds.

In [92]:
# takes in a dictionary of worlds, and spits out next state of worlds
def roll_the_dice(worlds):
    next_worlds = dict()
    for world in worlds:
        num_worlds = worlds[world]

        if world == 'p1 wins' or world == 'p2 wins':
            if world in next_worlds:
                next_worlds[world] += num_worlds
            else:
                next_worlds[world] = num_worlds
            continue
        
        curr_p1_position, curr_p2_position, curr_p1_score, curr_p2_score, curr_turn = world
        next_p1_position, next_p2_position, next_p1_score, next_p2_score, next_turn = world
        
        
        if curr_turn == 'p1':
            for i in range (1,4):
                for j in range(1,4):
                    for k in range(1,4):
                        next_p1_position, next_p2_position, next_p1_score, next_p2_score = (curr_p1_position, curr_p2_position, curr_p1_score, curr_p2_score)
                        next_turn = 'p2'

                        next_p1_position += i+j+k

                        #I didn't realize I had a much cleaner implementation
                        next_p1_position = next_p1_position-10 if next_p1_position>10 else next_p1_position
                        next_p1_score += next_p1_position

                        next_world = (next_p1_position, next_p2_position, next_p1_score, next_p2_score, next_turn)

                        if next_p1_score >= 21:
                            if 'p1 wins' in next_worlds:
                                next_worlds['p1 wins'] += num_worlds
                            else:
                                next_worlds['p1 wins'] = num_worlds
                            continue

                        if next_world in next_worlds:
                            next_worlds[next_world] += num_worlds
                        else:
                            next_worlds[next_world] = num_worlds
        elif curr_turn == 'p2':
            for i in range(1,4):
                for j in range(1,4):
                        for k in range(1,4):
                            next_p1_position, next_p2_position, next_p1_score, next_p2_score = (curr_p1_position, curr_p2_position, curr_p1_score, curr_p2_score)
                            next_turn = 'p1'

                            next_p2_position += i+j+k

                            #I didn't realize I had a much cleaner implementation
                            next_p2_position = next_p2_position-10 if next_p2_position>10 else next_p2_position
                            next_p2_score += next_p2_position

                            next_world = (next_p1_position, next_p2_position, next_p1_score, next_p2_score, next_turn)

                            if next_p2_score >= 21:
                                if 'p2 wins' in next_worlds:
                                    next_worlds['p2 wins'] += num_worlds
                                else:
                                    next_worlds['p2 wins'] = num_worlds
                                continue

                            if next_world in next_worlds:
                                next_worlds[next_world] += num_worlds
                            else:
                                next_worlds[next_world] = num_worlds
                    
    return next_worlds

In [93]:
current_worlds = worlds
while len(current_worlds)!=2:
    current_worlds = roll_the_dice(current_worlds)

In [94]:
current_worlds

{'p1 wins': 146854918035875, 'p2 wins': 143358457407936}

In [98]:
max(current_worlds.values())

146854918035875